# Test case 2D-2 (unsteady)

In [4]:
import numpy as np
from collections import defaultdict
from pycutfem.utils.meshgen import structured_quad
from pycutfem.core.mesh import Mesh
from pycutfem.core.levelset import AffineLevelSet
from pycutfem.fem.mixedelement import MixedElement
from pycutfem.core.dofhandler import DofHandler
from pycutfem.ufl.compilers import FormCompiler
from pycutfem.ufl.expressions import Function, Jump, Hessian, restrict, Derivative
from pycutfem.integration.quadrature import line_quadrature
from pycutfem.ufl.helpers import HelpersFieldAware as _hfa, _collapsed_hess

poly_order = 2
L, H = 2.0, 1.0
nx, ny = 20, 5
nodes, elems, edges, corners = structured_quad(L, H, nx=nx, ny=ny, poly_order=poly_order)
mesh = Mesh(nodes, elems, edges, corners, element_type='quad', poly_order=poly_order)
ls = AffineLevelSet(a=1.0, b=0, c=-1.0)
mesh.classify_elements(ls)
mesh.build_interface_segments(ls)
mesh.classify_edges(ls)

me = MixedElement(mesh, field_specs={'u': poly_order})
dh = DofHandler(me, method='cg')
comp = FormCompiler(dh)

has_pos = mesh.element_bitset('cut') | mesh.element_bitset('outside')
has_neg = mesh.element_bitset('cut') | mesh.element_bitset('inside')

u_pos = Function('u_pos', 'u', dh, side='+')
u_pos.set_values_from_function(lambda x, y: np.where(x > 1.0, (x - 1.0) ** 2, 0.0))
u_neg = Function('u_neg', 'u', dh, side='-')
u_neg.set_values_from_function(lambda x, y: np.where(x > 1.0, 0.0,(x - 1.0) ** 2))
# r_pos = restrict(u_pos, has_pos)
# r_neg = restrict(u_neg, has_neg)
r_pos = u_pos
r_neg = u_neg
print(f"num of ghost_pos edges {mesh.edge_bitset('ghost_pos').cardinality()}")
edge_id = int(mesh.edge_bitset('ghost_pos').to_indices()[0])
e = mesh.edge(edge_id)
pos_eid, neg_eid = e.right, e.left

pos_dofs = dh.get_elemental_dofs(pos_eid)
neg_dofs = dh.get_elemental_dofs(neg_eid)
print(f"edge {edge_id} between elements {pos_eid} (+) and {neg_eid} (-)")
print(f"pos dofs {pos_dofs}")
print(f"neg dofs {neg_dofs}")
common_dofs = np.intersect1d(pos_dofs, neg_dofs)
print(f"common dofs {common_dofs}")
union = np.unique(np.concatenate([pos_dofs, neg_dofs]))
pos_map = np.searchsorted(union, pos_dofs)
neg_map = np.searchsorted(union, neg_dofs)
print(f"pos_map {pos_map}")
print(f"neg_map {neg_map}")
common_map = np.intersect1d(pos_map, neg_map)
print(f"common map {common_map}")

pos_map_by_field = {'u': np.asarray(np.searchsorted(union, _hfa.elemental_field_dofs(dh, int(pos_eid), 'u')), dtype=int)}
neg_map_by_field = {'u': np.asarray(np.searchsorted(union, _hfa.elemental_field_dofs(dh, int(neg_eid), 'u')), dtype=int)}

print(f"pos_map_by_field {pos_map_by_field}")
print(f"neg_map_by_field {neg_map_by_field}")

p0, p1 = mesh.nodes_x_y_pos[list(e.nodes)]
print(f"edge {edge_id} nodes {e.nodes} at {p0}, {p1}")
qpts, _ = line_quadrature(p0, p1, 4)
mid = qpts[0]

comp.ctx.update({
    'basis_values': {'+': defaultdict(dict), '-': defaultdict(dict)},
    'global_dofs': union,
    'pos_map': pos_map,
    'neg_map': neg_map,
    'pos_map_by_field': pos_map_by_field,
    'neg_map_by_field': neg_map_by_field,
    'coeff_mask_pos_global': np.ones(len(union)),
    'coeff_mask_neg_global': np.ones(len(union)),
    'pos_eid': pos_eid,
    'neg_eid': neg_eid,
    'x_phys': mid,
    'x_phys_pos': mid,
    'x_phys_neg': mid,
    'normal': np.array([1.0, 0.0]),
    'is_ghost': True,
    'mask_basis': True,
    'use_union_local_dofs': True,
    '_ghost_level_set': ls,
    'rhs': False
})
def jump(expr_pos, expr_neg):    
    comp.ctx['phi_val'] = 1.0
    comp.ctx['side'] = '+'
    comp.ctx['eid'] = pos_eid
    pos_h = comp._visit(expr_pos)
    comp.ctx['phi_val'] = -1.0
    comp.ctx['side'] = '-'
    comp.ctx['eid'] = neg_eid
    neg_h = comp._visit(expr_neg)
    return pos_h, neg_h
def _hess_comp(a, b):
    return (Derivative(a,2,0)*Derivative(b,2,0) +
            2*Derivative(a,1,1)*Derivative(b,1,1) +
            Derivative(a,0,2)*Derivative(b,0,2))
def _hess_single(a):
    return (Derivative(a,2,0) +
            2*Derivative(a,1,1) +
            Derivative(a,0,2))
pos_h, neg_h = jump(Hessian(r_pos), Hessian(r_neg))
deriv_pos_h, deriv_neg_h = jump(_hess_single(r_pos), _hess_single(r_neg))
print('pos deriv hess', deriv_pos_h.data.sum())
print('neg deriv hess', deriv_neg_h.data.sum())
print('pos hess', pos_h.data)
print('neg hess', neg_h.data)

print('pos collapsed', _collapsed_hess(pos_h))
print('neg collapsed', _collapsed_hess(neg_h))
print('difference', _collapsed_hess(pos_h) - _collapsed_hess(neg_h))

num of ghost_pos edges 5
edge 32 between elements 11 (+) and 10 (-)
pos dofs [64 69 70 66 71 72 68 73 74]
neg dofs [58 63 64 60 65 66 62 67 68]
common dofs [64 66 68]
pos_map [ 4  9 10  6 11 12  8 13 14]
neg_map [0 3 4 1 5 6 2 7 8]
common map [4 6 8]
pos_map_by_field {'u': array([ 4,  9, 10,  6, 11, 12,  8, 13, 14])}
neg_map_by_field {'u': array([0, 3, 4, 1, 5, 6, 2, 7, 8])}
edge 32 nodes (22, 104) at [1.1 0. ], [1.1 0.2]
pos deriv hess 1.999999999999991
neg deriv hess 0.0
pos hess [[[ 2.00000000e+00 -2.03770838e-14]
  [-2.03770838e-14 -3.84886567e-15]]]
neg hess [[[0. 0.]
  [0. 0.]]]
pos collapsed [[[ 2.00000000e+00 -2.03770838e-14]
  [-2.03770838e-14 -3.84886567e-15]]]
neg collapsed [[[0. 0.]
  [0. 0.]]]
difference [[[ 2.00000000e+00 -2.03770838e-14]
  [-2.03770838e-14 -3.84886567e-15]]]
